<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/save_huggingface_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install timm
!pip3 install onnx-simplifier

from transformers import DetrFeatureExtractor, DetrForObjectDetection
from PIL import Image
import requests
import torch
import torch.onnx

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes

print(logits, bboxes)

name = 'detr_resnet-50.onnx'
inputs['pixel_mask'].shape
dummy_input = torch.ones(1, 3, 800, 1066, dtype=torch.float)
torch.onnx.export(model, dummy_input, name, verbose=True,opset_version=11)
!python3 -m onnxsim 'detr_resnet-50.onnx' 'detr_resnet-50-simplified.onnx'